In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import time

In [2]:
batch_size = 32
learning_rate = 1e-3
num_epochs = 100

train_dataset = datasets.MNIST(
    root='./data', train=True, transform=transforms.ToTensor(), download=True)

test_dataset = datasets.MNIST(
    root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Processing...
Done!


In [16]:
class LogisticRegression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(LogisticRegression, self).__init__()
        self.logistic = nn.Linear(in_dim, n_class)
        
    def forward(self, x):
        out = self.logistic(x)
        return out

model = LogisticRegression(28 * 28, 10)
use_gpu = torch.cuda.is_available()  # 判断是否有gpu
if use_gpu:
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    since = time.time()
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.view(img.size(0), -1)
        
        if use_gpu:
            img = Variable(img).cuda()
            label = Variable(label).cuda()
        else:
            img = Variable(img)
            label = Variable(label)
            
        out = model(img)
        loss = criterion(out, label)  # 得到的是batch_size的均值
       
        running_loss += loss.data[0] * label.size(0)  # 因此乘以batch_size的值得到总得损失
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum().float()  # 记得将整数转换成浮点数
    
        running_acc += num_correct.data[0]
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 300 == 0:
            
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                epoch + 1, num_epochs, running_loss / (batch_size * i),
                running_acc / (batch_size * i)))
    print('Finish {} epoch, Loss: {:.6f}, Acc: {:.6f}'.format(
        epoch + 1, running_loss / (len(train_dataset)), running_acc / (len(
            train_dataset))))
    
    model.eval()
    eval_loss = 0.
    eval_acc = 0.
    for data in test_loader:
        img, label = data
        img = img.view(img.size(0), -1)
        if torch.cuda.is_available():
            img = Variable(img, volatile=True).cuda()
            label = Variable(label, volatile=True).cuda()
        else:
            img = Variable(img, volatile=True)
            label = Variable(label, volatile=True)
        out = model(img)
        loss = criterion(out, label)
        eval_loss += loss.data[0] * label.size(0)
        _, pred = torch.max(out, 1)
        num_correct = (pred == label).sum()
        eval_acc += num_correct.data[0]
    print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(
        test_dataset)), eval_acc / (len(test_dataset))))

/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:36: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[1/100] Loss: 2.152924, Acc: 0.361458
[1/100] Loss: 2.023442, Acc: 0.497552
[1/100] Loss: 1.913465, Acc: 0.576736
[1/100] Loss: 1.815879, Acc: 0.623516
[1/100] Loss: 1.735123, Acc: 0.653854
[1/100] Loss: 1.661791, Acc: 0.676823
Finish 1 epoch, Loss: 1.644873, Acc: 0.682100


/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:62: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:63: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:69: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/jiangxinyang848/anaconda3/envs/jiang/lib/python3.5/site-packages/ipykernel_launcher.py:72: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Test Loss: 1.207101, Acc: 0.000000
[2/100] Loss: 1.185169, Acc: 0.808958
[2/100] Loss: 1.151852, Acc: 0.810885
[2/100] Loss: 1.123828, Acc: 0.813438
[2/100] Loss: 1.095413, Acc: 0.815182
[2/100] Loss: 1.070257, Acc: 0.815521
[2/100] Loss: 1.045861, Acc: 0.817587
Finish 2 epoch, Loss: 1.040693, Acc: 0.817950
Test Loss: 0.881139, Acc: 0.000000
[3/100] Loss: 0.886747, Acc: 0.831875
[3/100] Loss: 0.873669, Acc: 0.833490
[3/100] Loss: 0.860555, Acc: 0.834896
[3/100] Loss: 0.849261, Acc: 0.835964
[3/100] Loss: 0.837757, Acc: 0.837125
[3/100] Loss: 0.828220, Acc: 0.837066
Finish 3 epoch, Loss: 0.825415, Acc: 0.837417
Test Loss: 0.736072, Acc: 0.000000
[4/100] Loss: 0.746437, Acc: 0.845417
[4/100] Loss: 0.747209, Acc: 0.844583
[4/100] Loss: 0.739671, Acc: 0.844861
[4/100] Loss: 0.732937, Acc: 0.845182
[4/100] Loss: 0.723611, Acc: 0.847875
[4/100] Loss: 0.717596, Acc: 0.848611
Finish 4 epoch, Loss: 0.716568, Acc: 0.849017
Test Loss: 0.653585, Acc: 0.000000
[5/100] Loss: 0.676940, Acc: 0.851458


[28/100] Loss: 0.391737, Acc: 0.897083
[28/100] Loss: 0.393951, Acc: 0.894479
[28/100] Loss: 0.391905, Acc: 0.895313
[28/100] Loss: 0.391830, Acc: 0.895885
[28/100] Loss: 0.388551, Acc: 0.896458
[28/100] Loss: 0.391527, Acc: 0.895139
Finish 28 epoch, Loss: 0.391035, Acc: 0.895267
Test Loss: 0.369750, Acc: 0.000000
[29/100] Loss: 0.385354, Acc: 0.896458
[29/100] Loss: 0.383085, Acc: 0.897708
[29/100] Loss: 0.386871, Acc: 0.896875
[29/100] Loss: 0.389925, Acc: 0.896016
[29/100] Loss: 0.387681, Acc: 0.895542
[29/100] Loss: 0.386538, Acc: 0.896233
Finish 29 epoch, Loss: 0.388257, Acc: 0.895683
Test Loss: 0.367393, Acc: 0.000000
[30/100] Loss: 0.376881, Acc: 0.897604
[30/100] Loss: 0.379910, Acc: 0.897188
[30/100] Loss: 0.380369, Acc: 0.897778
[30/100] Loss: 0.383513, Acc: 0.897500
[30/100] Loss: 0.384279, Acc: 0.896896
[30/100] Loss: 0.385803, Acc: 0.896493
Finish 30 epoch, Loss: 0.385646, Acc: 0.896250
Test Loss: 0.364906, Acc: 0.000000
[31/100] Loss: 0.396872, Acc: 0.892396
[31/100] Loss

[54/100] Loss: 0.355813, Acc: 0.903125
[54/100] Loss: 0.353155, Acc: 0.902813
[54/100] Loss: 0.351870, Acc: 0.902639
[54/100] Loss: 0.350746, Acc: 0.903047
[54/100] Loss: 0.348101, Acc: 0.903687
[54/100] Loss: 0.347472, Acc: 0.904115
Finish 54 epoch, Loss: 0.346988, Acc: 0.904300
Test Loss: 0.330336, Acc: 0.000000
[55/100] Loss: 0.339129, Acc: 0.907188
[55/100] Loss: 0.346438, Acc: 0.903854
[55/100] Loss: 0.342700, Acc: 0.904722
[55/100] Loss: 0.343843, Acc: 0.904661
[55/100] Loss: 0.346704, Acc: 0.903938
[55/100] Loss: 0.344910, Acc: 0.905017
Finish 55 epoch, Loss: 0.345968, Acc: 0.904517
Test Loss: 0.329423, Acc: 0.000000
[56/100] Loss: 0.343804, Acc: 0.906771
[56/100] Loss: 0.351862, Acc: 0.903229
[56/100] Loss: 0.351144, Acc: 0.903993
[56/100] Loss: 0.347907, Acc: 0.904505
[56/100] Loss: 0.343809, Acc: 0.905000
[56/100] Loss: 0.344137, Acc: 0.905087
Finish 56 epoch, Loss: 0.344949, Acc: 0.904583
Test Loss: 0.328660, Acc: 0.000000
[57/100] Loss: 0.347274, Acc: 0.904583
[57/100] Loss

[80/100] Loss: 0.327785, Acc: 0.907604
[80/100] Loss: 0.335002, Acc: 0.905365
[80/100] Loss: 0.331815, Acc: 0.906771
[80/100] Loss: 0.329934, Acc: 0.908412
[80/100] Loss: 0.330888, Acc: 0.907896
[80/100] Loss: 0.326087, Acc: 0.909201
Finish 80 epoch, Loss: 0.326961, Acc: 0.909117
Test Loss: 0.313072, Acc: 0.000000
[81/100] Loss: 0.338083, Acc: 0.907292
[81/100] Loss: 0.332617, Acc: 0.907760
[81/100] Loss: 0.331670, Acc: 0.907604
[81/100] Loss: 0.329171, Acc: 0.908490
[81/100] Loss: 0.327098, Acc: 0.909354
[81/100] Loss: 0.326363, Acc: 0.909583
Finish 81 epoch, Loss: 0.326388, Acc: 0.909550
Test Loss: 0.312588, Acc: 0.000000
[82/100] Loss: 0.325634, Acc: 0.908542
[82/100] Loss: 0.327863, Acc: 0.908698
[82/100] Loss: 0.329245, Acc: 0.909444
[82/100] Loss: 0.325561, Acc: 0.910208
[82/100] Loss: 0.326370, Acc: 0.910042
[82/100] Loss: 0.325022, Acc: 0.909826
Finish 82 epoch, Loss: 0.325810, Acc: 0.909517
Test Loss: 0.312082, Acc: 0.000000
[83/100] Loss: 0.325699, Acc: 0.913854
[83/100] Loss